In [1]:
import os
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import lasio
import tkinter as tk
from tkinter import filedialog, messagebox
import webbrowser
from tempfile import NamedTemporaryFile

In [2]:
class VisualizadorLogPozoPlotly:
    def __init__(self):
        self.df = None
        self.columnas = []
        self.col_profundidad = None
        self.cols_seleccionadas = []
        self.tipo_archivo = None
        
        # Colores predefinidos para las curvas
        self.colores = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', 
                        '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
    
    def procesar_csv(self, archivo):
        """Procesa un archivo CSV y lo carga en un DataFrame"""
        try:
            # Leer archivo CSV con pandas
            self.df = pd.read_csv(archivo, sep=';' )
            
            # Actualizar columnas disponibles
            self.columnas = list(self.df.columns)
            
            # Autodetectar columna de profundidad
            posibles_prof = ['DEPTH', 'PROF', 'PROFUNDIDAD', 'MD', 'TVD', 'DEPTH_M', 'DEPTH_FT']
            
            self.col_profundidad = next((col for col in self.columnas 
                                        if col.upper() in posibles_prof), self.columnas[0])
            
            # Seleccionar primeras 5 columnas como predeterminadas (excluyendo profundidad)
            cols_disponibles = [col for col in self.columnas if col != self.col_profundidad]
            self.cols_seleccionadas = cols_disponibles[:5]
            
            return True
            
        except Exception as e:
            print(f"Error al procesar CSV: {str(e)}")
            return False
    
    def procesar_las(self, archivo):
        """Procesa un archivo LAS y lo carga en un DataFrame"""
        try:
            # Usar lasio para leer archivo LAS
            las = lasio.read(archivo)
            
            # Convertir a DataFrame
            self.df = las.df()
            
            # Restablecer índice para tener profundidad como columna
            self.df = self.df.reset_index()
            
            # Renombrar la columna de profundidad si es necesario
            if 'DEPT' in self.df.columns:
                self.df = self.df.rename(columns={'DEPT': 'DEPTH'})
            
            # Actualizar columnas disponibles
            self.columnas = list(self.df.columns)
            
            # Columna de profundidad es siempre la primera en archivos LAS
            self.col_profundidad = self.columnas[0]
            
            # Seleccionar primeras 5 columnas como predeterminadas (excluyendo profundidad)
            cols_disponibles = [col for col in self.columnas if col != self.col_profundidad]
            self.cols_seleccionadas = cols_disponibles[:5]
            
            return True
            
        except Exception as e:
            print(f"Error al procesar LAS: {str(e)}")
            return False
    
    def crear_visualizacion(self, mostrar=True, guardar_html=None):
        """Crea una visualización de los logs con Plotly"""
        if self.df is None or len(self.cols_seleccionadas) == 0:
            print("No hay datos para visualizar")
            return None
        
        # Crear figura de subplots
        fig = make_subplots(
            rows=1, 
            cols=len(self.cols_seleccionadas),
            shared_yaxes=True,
            horizontal_spacing=0.02
        )
        
        # Obtener datos de profundidad
        prof = self.df[self.col_profundidad].values
        
        # Añadir cada curva seleccionada
        for i, col in enumerate(self.cols_seleccionadas):
            color = self.colores[i % len(self.colores)]
            
            # Crear una traza para la curva actual
            fig.add_trace(
                go.Scatter(
                    x=self.df[col].values,
                    y=prof,
                    name=col,
                    line=dict(color=color, width=2),
                    hovertemplate=f"{col}: %{{x}}<br>{self.col_profundidad}: %{{y}}<extra></extra>"
                ),
                row=1, col=i+1
            )
            
            # Configurar el eje X
            fig.update_xaxes(
                title_text=col,
                title_font=dict(color=color),
                tickfont=dict(color=color),
                showgrid=True,
                zeroline=True,
                row=1, col=i+1
            )
        
        # Configurar el eje Y (profundidad)
        fig.update_yaxes(
            title_text=self.col_profundidad,
            autorange="reversed",  # Invertir eje Y para que la profundidad aumente hacia abajo
            showgrid=True,
            zeroline=True,
            row=1, col=1
        )
        
        # Configuración general
        fig.update_layout(
            title=f"Visualización de Logs de Pozo",
            height=800,
            width=200 * len(self.cols_seleccionadas) + 100,
            template="plotly_white",
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1
            ),
            margin=dict(l=50, r=50, t=50, b=50)
        )
        
        # Guardar en HTML si se especifica
        if guardar_html:
            fig.write_html(guardar_html)
        
        # Mostrar la figura
        if mostrar:
            fig.show()
        
        return fig

In [3]:
def seleccionar_y_visualizar():
    """Función principal para seleccionar archivo y visualizar"""
    # Crear instancia del visualizador
    visualizador = VisualizadorLogPozoPlotly()
    
    # Crear ventana raíz de Tkinter (será ocultada)
    root = tk.Tk()
    root.withdraw()
    
    # Solicitar archivo
    archivo = filedialog.askopenfilename(
        title="Seleccionar archivo CSV o LAS",
        filetypes=[("Archivos de log", "*.csv;*.las"), ("Todos los archivos", "*.*")]
    )
    
    if not archivo:
        print("No se seleccionó ningún archivo")
        root.destroy()
        return
    
    # Determinar tipo de archivo
    extension = os.path.splitext(archivo)[1].lower()
    
    # Procesar archivo según su tipo
    if extension == '.csv':
        exito = visualizador.procesar_csv(archivo)
    elif extension == '.las':
        exito = visualizador.procesar_las(archivo)
    else:
        messagebox.showerror("Error", "Formato de archivo no soportado. Use .csv o .las")
        root.destroy()
        return
    
    if not exito:
        messagebox.showerror("Error", "Error al procesar el archivo")
        root.destroy()
        return
    
    # Crear archivo HTML temporal
    temp_html = NamedTemporaryFile(delete=False, suffix='.html')
    temp_html.close()
    
    # Generar visualización y guardarla
    visualizador.crear_visualizacion(mostrar=False, guardar_html=temp_html.name)
    
    # Abrir en el navegador
    webbrowser.open('file://' + temp_html.name)
    
    # Mostrar información
    print(f"Archivo procesado: {os.path.basename(archivo)}")
    print(f"Columna de profundidad: {visualizador.col_profundidad}")
    print(f"Curvas visualizadas: {', '.join(visualizador.cols_seleccionadas)}")
    
    # Destruir ventana raíz
    root.destroy()


In [4]:
def visualizar_archivo(ruta_archivo, cols_a_visualizar=None, col_profundidad=None, guardar_html=None):
    """
    Función para visualizar directamente un archivo sin interfaz gráfica
    
    Parámetros:
    - ruta_archivo: Ruta al archivo CSV o LAS
    - cols_a_visualizar: Lista de columnas a visualizar (opcional)
    - col_profundidad: Nombre de la columna de profundidad (opcional)
    - guardar_html: Ruta donde guardar el HTML generado (opcional)
    
    Retorna:
    - Figura de Plotly
    """
    visualizador = VisualizadorLogPozoPlotly()
    
    # Determinar tipo de archivo
    extension = os.path.splitext(ruta_archivo)[1].lower()
    
    # Procesar archivo según su tipo
    if extension == '.csv':
        exito = visualizador.procesar_csv(ruta_archivo)
    elif extension == '.las':
        exito = visualizador.procesar_las(ruta_archivo)
    else:
        print("Formato de archivo no soportado. Use .csv o .las")
        return None
    
    if not exito:
        print("Error al procesar el archivo")
        return None
    
    # Establecer columna de profundidad personalizada si se proporciona
    if col_profundidad and col_profundidad in visualizador.columnas:
        visualizador.col_profundidad = col_profundidad
    
    # Establecer columnas a visualizar personalizadas si se proporcionan
    if cols_a_visualizar:
        cols_validas = [col for col in cols_a_visualizar if col in visualizador.columnas]
        if cols_validas:
            visualizador.cols_seleccionadas = cols_validas
    
    # Crear visualización
    fig = visualizador.crear_visualizacion(mostrar=True, guardar_html=guardar_html)
    
    return fig

In [6]:
if __name__ == "__main__":
    # Ejemplo de uso directo
    # visualizar_archivo("ruta/a/tu/archivo.las")
    
    # Ejemplo con interfaz para selección de archivo
    seleccionar_y_visualizar()

Archivo procesado: V624v2.csv
Columna de profundidad: DEPT
Curvas visualizadas: AT20, P_IMPEDANCE, S_IMPEDANCE, Vp/Vs, zonas
